<table width="100%">
    <td align="left">
        <a target="_blank", href="https://www.up.pt/fcup/en/">
            <img src="https://divulgacao.iastro.pt/wp-content/uploads/2023/03/FCUP_logo-print_blcktransp_600ppi.png" width="90px" height="90px" style="padding-bottom:5px;"/>
        </a>
    </td>
    <td>
        <a target="_blank", href="https://www.iastro.pt/">
            <img src="https://divulgacao.iastro.pt/wp-content/uploads/2018/03/IA_logo_bitmap-rgbblack-1200px-388x259.png" width="90px" height="90px" style="padding-bottom:5px;"/>
        </a>
    </td>
    <td align="center">
        <a target="_blank" href="https://colab.research.google.com/github/jbrinchmann/MLD2025/blob/main/Notebooks/MLD2025-01c-Python%20and%20SQL.ipynb">
           <img src="https://tinyurl.com/3mm2cyk6"  width="90px" height="90px" style="padding-bottom:5px;"/>Run in Google Colab
        </a>
    </td>
<td align="center"><a target="_blank" href="https://github.com/jbrinchmann/MLD2025/blob/main/Notebooks/MLD2025-01c-Python%20and%20SQL.ipynb">
<img src="https://tinyurl.com/25h5fw53"  width="90px" height="60px" style="padding-bottom:0px;"  />View Source on GitHub</a></td>
</table>

# Python and SQL

Here we will try a couple of ways to query a database with Python.  We will start with the built-in option in python - `sqlite3`. You should always have this available and it is lightweight. In order to make use of this, we need a simple database - in the lecture notes, or doing the first exercise you can create this file. If you are on Google Colab you need to copy this file into your runtime so the first line does that:



In [12]:
#!wget --quiet -O MLD2025.db https://github.com/jbrinchmann/MLD2025/blob/main/ProblemSets/MakeTables/MLD2025.db

In [13]:
import sqlite3 as lite

In [14]:
con = lite.connect('../MLD2025.db')

We now have a connection to the database. We can launch queries through this to search the database. Here are some examples

## Getting an overview of a database

In our case here we have the code (if you look a bit) for how the database was created, but sometimes you are not as lucky. In that case there is usually a way to get a list of the tables etc - for sqlite this is done as follows to get the tables:

In [15]:
with con:
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print(cur.fetchall())

[('Stars',), ('Observations',)]


If instead you want to see the SQL used to create the table you can do:

In [27]:
with con:
    cur = con.cursor()
    cur.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='Stars';")
    lines = cur.fetchall()
print(lines[0][0]) # Just to get better formatting...

CREATE TABLE Stars (StarID INT,
    			 FieldID INT, Star varchar(10), ra DOUBLE,
    			 decl DOUBLE, g FLOAT, r FLOAT,
    			 UNIQUE(StarID), PRIMARY KEY(StarID),
    			 FOREIGN KEY(FieldID) REFERENCES Observations(ID))


## Exploring the tables

Let us now do something more useful - for a first try, we will just print out the stars that are in the Stars table. This is easily done as follows:

In [17]:
with con:
    # Get a cursor.
    cur = con.cursor()

    query = "SELECT star FROM stars" 
    # Execute commands
    rows = cur.execute(query)
    print(rows.fetchall())

[('S1',), ('S2',), ('S5',), ('S7',)]


Note that the case does not matter - I asked for table `stars` even though the table is called `Stars` above - same thing. Likewise, I have capitalized the commands `SELECT` and `FROM` but that is again case-insensitive.

I used `fetchall` to get all rows above, but the output is not very pretty - so it is usually better to take more control over the fetching and for large databases this is essential! Here is a slightly more involved example which joins the Stars and Observations tables and then loops over the results to print the results more nicely:

In [18]:
with con:
    # Get a cursor.
    cur = con.cursor()

    query = "select s.star, o.Date from stars as s JOIN observations as o ON s.fieldID=o.ID" 
    # Execute commands
    rows = cur.execute(query)

The `rows` variable now contains a cursor object which points to the results table. To get values out from it we need to loop over it.

In [19]:
for row in rows:
    this_star, this_date = row
    print("Star = {0} was observed  at {1}".format(this_star, this_date))

Star = S1 was observed  at 92.9885764
Star = S2 was observed  at 92.9885764
Star = S5 was observed  at 93.5532134
Star = S7 was observed  at 97.3323764


**Try:** run the cell above again. What happened? Why?

### Task:

Now try to use the preceding to answer this question:

- Where is the FITS image stored for star S5?

In [28]:
with con:
    cur = con.cursor()
    query = "SELECT s.star, o.WhereStored FROM stars AS s JOIN observations as o ON s.fieldID=o.ID WHERE s.star='S5'"
    rows = cur.execute(query)
    print(rows.fetchall())

[('S5', '/disks/yaeps-1/StF-045.fits')]


# Python and SQL - the Pandas way

The approach above using `sqlite3` will always work regardless of the python installation but it is a bit clunky so you might wonder whether a better way exists, and in a number of contexts the answer is yes. 

This easier way is provided by [pandas](https://pandas.pydata.org/), a fairly heavy but powerful package to interact with tabular data of various types. It is widely used in both academia and industry and hence is a useful tool to know. However this might not be the best approach if your database is very large - caveat emptor.

In [20]:
import pandas as pd

In [21]:
con = lite.connect("../MLD2025.db")

Let us now do a slightly varianty on the search for stars earlier - this time getting the Ra and Dec from the Stars table:

In [22]:
t = pd.read_sql_query('Select ra, decl from Stars', con) 

In [23]:
t

,ra,decl
0,198.847500,10.503472
1,198.565417,11.023194
2,198.937083,9.916889
3,199.251667,10.348694


That is simpler of course. It boils down to writing the same SQL code that you executed above and then read the results with the `read_sql_query` function and you get the results nicely in a pandas dataframe which prints nicely and which is easy to then use for processing.

### Task: 

Now use the pandas approach to answer this question:

- Give me a list of all stars observed with the same FieldID

In [24]:
for fieldID in (1, 2, 3):
    query = 'Select * from Stars where FieldID={0}'.format(fieldID)
    t = pd.read_sql_query(query, con)
    print("\nFor Field ID = {0} we have the following stars: ".format(fieldID))
    print(t)


For Field ID = 1 we have the following stars: 
   StarID  FieldID Star          ra       decl     g     r
0       1        1   S1  198.847500  10.503472  14.5  15.2
1       2        1   S2  198.565417  11.023194  15.3  15.4

For Field ID = 2 we have the following stars: 
   StarID  FieldID Star          ra       decl     g     r
0       4        2   S7  199.251667  10.348694  14.6  14.1

For Field ID = 3 we have the following stars: 
   StarID  FieldID Star          ra      decl     g     r
0       3        3   S5  198.937083  9.916889  16.4  15.8
